<a href="https://colab.research.google.com/github/python-freak/Sentiment-Analysis/blob/master/sentiment_Analysis_NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/python-freak/Sentiment-Analysis

Cloning into 'Sentiment-Analysis'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = "https://drive.google.com/open?id=1q1-dZFkdXZ19vUn3-u6HGAm0df2BH50G"
fluff, id = link.split('=')
print(id)

1q1-dZFkdXZ19vUn3-u6HGAm0df2BH50G


In [0]:
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('glove.6B.100d.txt')

In [0]:
! pwd
%cd ../
! pwd

/content
/
/


In [0]:
%cd .content/Sentiment-Analysis

[Errno 2] No such file or directory: '.content/Sentiment-Analysis'
/


In [0]:
! ls

bin	    content  etc   lib32  mnt	root  srv    tmp    var
boot	    datalab  home  lib64  opt	run   swift  tools
colabtools  dev      lib   media  proc	sbin  sys    usr


In [0]:
%cd ./content
! ls
%cd ./Sentiment-Analysis
! ls

/content
adc.json  glove.6B.100d.txt  sample_data  Sentiment-Analysis
/content/Sentiment-Analysis
download.py  imdb.py


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf 
import numpy as np
from scipy.spatial.distance import cdist

In [0]:
from tensorflow.python.keras.models import Sequential , Model
from tensorflow.python.keras.layers import Dense , GRU , Embedding , Input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras import regularizers
from keras.initializers import Constant
import os

Using TensorFlow backend.


In [0]:
import imdb

In [0]:
imdb.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [0]:
x_train_text , y_train = imdb.load_data(train = True)
x_test_text  , y_test  = imdb.load_data(train = False)

In [0]:
print("Train-set size: ",len(x_train_text))
print("Test-set size: " , len(x_test_text))

Train-set size:  25000
Test-set size:  25000


In [0]:
x_train_text[3]

"Steven Speilberg's adaptation of Alice Walkers popular novel is not without its share of controversy. When first released members of the black community criticised its treatment of black men, while others questioned why a white man was directing this film about black women.<br /><br />This is the story of a young black woman named Celie, growing up in rural America after the turn of the century. She has two children by her abusive father which are snatched from her arms at birth. Her only solace in her miserable life comes from her sister.<br /><br />Celie (played in later years by newcomer Whoopie Goldberg) is married off to an abusive husband (Danny Glover). The husband is humiliated by the sister and so she is quickly removed from Celie's life.<br /><br />The story is often heartbreaking as Celie keeps up hope that she may one day be reunited with her sister and with her children. Throughout her life she meets an assortment of characters, including Sophia, a tough as nails wife to 

In [0]:
y_train[3]

1.0

In [0]:
data_text = x_train_text + x_test_text

In [0]:
num_words = 10000

In [0]:
tokenizer = Tokenizer(num_words = num_words)

In [0]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 11.7 s, sys: 87.6 ms, total: 11.8 s
Wall time: 11.8 s


In [0]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

In [0]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

In [0]:
x_train_text[1]

'In the hands of a more skilled director, this film would have been considered a horror masterpiece. Despite Michael "Death Wish" Winner\'s merely passable direction, the movie is interesting, original and more than a little scary.<br /><br />The script bucks more than one horror cliché off its back (several it can\'t shake) including Chris Sarandon as the heroine\'s boyfriend who actually listens to her as she insists that eerie things are going down. Burgess Meredith is delightful as the lovably insane neighbor. Eva Gardner is haunting with a young Beverly D\'Angelo as her mute and disturbed lesbian lover. John Carradine does a heck-of-a job sitting in a chair. And watch out for a brief cameo from an unknown-at-the-time Chris Walken! This movie is creepy and creative. The plot twists are lovely, if a tad predictable. The climax, of which I will give no detail, is disturbing and quite impressive. Again a better director could have done more with it, nonetheless it is quite satisfying 

In [0]:
np.array(x_train_tokens[1])

array([   8,    1,  915,    4,    3,   51, 6813,  164,   11,   19,   58,
         25,   75, 1228,    3,  182,  922,  467,  498,  327,  641, 1574,
       4679,  459,    1,   17,    6,  218,  212,    2,   51,   71,    3,
        120,  668,    7,    7,    1,  227, 3325,   51,   71,   27,  182,
       1515,  122,   92,  141,  439,    9,  185, 4423,  585, 1369, 5276,
         14,    1, 1339,   35,  160,    5,   40,   14,   59, 5719,   12,
       3461,  177,   23,  166,  175, 8230,    6, 2150,   14,    1, 2015,
       3777, 4061, 9842,    6, 2415,   16,    3,  186, 5422,   14,   40,
       4863,    2, 3949, 2372, 1493,  315, 4737,  124,    3, 2151,    4,
          3,  293, 1241,    8,    3, 3006,    2,  103,   41,   15,    3,
       1399, 2109,   36,   32, 1771,   30,    1,   55, 1369, 4086,   11,
         17,    6,  977,    2, 1527,    1,  111, 1258,   23, 1320,   43,
          3, 4446,  718,    1, 1391,    4,   60,   10,   80,  197,   54,
       1554,    6, 1218,    2,  179, 1186,  172,   

In [0]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [0]:
len(x_train_tokens)

25000

In [0]:
for tokens in x_train_tokens:
    print(len(tokens))
    print('\n')

61


248


168


303


146


353


592


96


131


423


166


127


419


329


146


147


662


230


137


118


374


956


140


247


231


139


434


124


233


141


112


40


112


116


275


136


121


69


356


109


126


197


327


227


148


378


107


221


234


75


139


289


191


192


124


181


214


363


640


316


120


100


164


193


321


139


162


64


774


332


547


137


142


158


571


122


206


130


216


130


117


204


627


167


271


161


182


246


262


496


156


76


122


150


115


472


289


173


234


348


164


255


115


110


96


266


232


47


467


119


122


173


63


406


217


201


357


212


244


220


114


338


56


239


54


189


183


177


193


271


145


714


152


188


119


77


152


279


891


167


78


100


234


193


129


139


111


318


94


155


241


159


53


127


111


72


118


95


84


99


495


183


306


471


123


123


221


53


624


140


9

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([ 61, 248, 168, ..., 197, 106, 160])

In [0]:
num_tokens.max()

2208

In [0]:
mean = num_tokens.mean()
mean

221.27716

In [0]:
std = num_tokens.std()
std

161.52716509099764

In [0]:
max_tokens = mean + 2*std
type(max_tokens)
max_tokens = int(max_tokens)
max_tokens

544

In [0]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94532

In [0]:
pad = 'pre'

In [0]:
x_train_pad = pad_sequences(x_train_tokens , maxlen = max_tokens , padding = pad , truncating = pad)

In [0]:
x_train_pad[50]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [0]:
x_test_pad = pad_sequences(x_test_tokens , maxlen = max_tokens , padding = pad , truncating = pad)

In [0]:
x_test_pad[1]

array([6442,  105, 2515,  290,   86, 9132, 3481,    2, 2154,   76,   82,
          3,  642, 3817,  300,    3, 2740, 1009, 2774,  140,    1,   19,
         60,  842,    5,   65,  625,  517,   46,  366,  222,  887, 8552,
        253,   31,  492,   17, 9794, 4188,  462,  783,   12,  669, 8983,
        172,    2,  462,  189,    3,   52, 1644,  163,   11,   55,   16,
       2253, 1106,    4,  265,    9,  589,   26,   43,   59,  154,   25,
         30,  224,   27,  129,  117,   59,  297,   16,  813,    2,   59,
        300,   27,    4, 2154,    2,  376,  135,    7,    7,  609,    2,
       2478,  422,    1, 1483,  324, 2478,  119,   53,   15,    3,  120,
       2660,   14,   69,    5,  609,    2, 5269,   25,  179,    3,  484,
        609,    2, 5269,  328,    5, 1315,   18, 5269, 3223,    9,  122,
          5,  408,    5,   11, 2458,  609,  270,   53,    5,    1,    5,
         76, 5269,  141,   18,   24, 1345, 5269,    1,    5,   60, 2820,
          5, 1194,   41,   20,   87,    2,  175,   

In [0]:
type(tokenizer.word_index)

dict

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values() , idx.keys()))
inverse_map

{1: 'the',
 2: 'and',
 3: 'a',
 4: 'of',
 5: 'to',
 6: 'is',
 7: 'br',
 8: 'in',
 9: 'it',
 10: 'i',
 11: 'this',
 12: 'that',
 13: 'was',
 14: 'as',
 15: 'for',
 16: 'with',
 17: 'movie',
 18: 'but',
 19: 'film',
 20: 'on',
 21: 'not',
 22: 'you',
 23: 'are',
 24: 'his',
 25: 'have',
 26: 'be',
 27: 'one',
 28: 'he',
 29: 'all',
 30: 'at',
 31: 'by',
 32: 'an',
 33: 'they',
 34: 'so',
 35: 'who',
 36: 'from',
 37: 'like',
 38: 'or',
 39: 'just',
 40: 'her',
 41: 'out',
 42: 'about',
 43: 'if',
 44: "it's",
 45: 'has',
 46: 'there',
 47: 'some',
 48: 'what',
 49: 'good',
 50: 'when',
 51: 'more',
 52: 'very',
 53: 'up',
 54: 'no',
 55: 'time',
 56: 'my',
 57: 'even',
 58: 'would',
 59: 'she',
 60: 'which',
 61: 'only',
 62: 'really',
 63: 'see',
 64: 'story',
 65: 'their',
 66: 'had',
 67: 'can',
 68: 'me',
 69: 'well',
 70: 'were',
 71: 'than',
 72: 'much',
 73: 'we',
 74: 'bad',
 75: 'been',
 76: 'get',
 77: 'do',
 78: 'great',
 79: 'other',
 80: 'will',
 81: 'also',
 82: 'into',
 83

In [0]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens]
    text = " ".join(words)
    return text

In [0]:
x_train_text[4]

'The Finnish version of Robert Altman\'s "Short Cuts", set in the small rural town of Äänekoski. The episodes present a kaleidoscope of the eternal events, problems and emotions of human life: joy and love, deception and disillusionment, hopelessness and death. I was particularly impressed by two episodes. The first is the story of the young waitress, who tries to stir up a romance for his co-workers: she radiates an overwhelming joy of love and life. The other episode presents and old man dying in a hospital and his wife trying to help him die with dignity. Particularly striking is the way the old couple have to fight their way against the humiliating practices of the hospital and their loneliness contrasted with the "routines" of the hospital crew (despite the signs of occasional empathy). Compared with Altman\'s classic, this movie is perhaps less professional, but it is definitely a great piece of art.'

In [0]:
tokens_to_string(x_train_tokens[4])

"the version of robert altman's short cuts set in the small rural town of the episodes present a of the eternal events problems and emotions of human life joy and love deception and and death i was particularly impressed by two episodes the first is the story of the young waitress who tries to stir up a romance for his co workers she an overwhelming joy of love and life the other episode presents and old man dying in a hospital and his wife trying to help him die with dignity particularly striking is the way the old couple have to fight their way against the humiliating practices of the hospital and their loneliness with the routines of the hospital crew despite the signs of occasional empathy compared with altman's classic this movie is perhaps less professional but it is definitely a great piece of art"

In [0]:
embedding_size = 100

In [0]:
embeddings_index = {}
f = open( './glove.6B.100d.txt' , encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs


print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
Num_words = min(num_words, len(idx)) + 1
embedding_matrix = np.zeros((Num_words, embedding_size))
print(embedding_matrix.shape)
for word, i in idx.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_size)


(10001, 100)


In [0]:
model = Sequential()

In [0]:
model.add(Embedding(Num_words,
                    embedding_size,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=max_tokens,
                    trainable = False))

NameError: ignored

In [0]:
model.add(Embedding(num_words,
                    embedding_size,
                    input_length=max_tokens,name = "layer_embedding"))

In [0]:
model.add(GRU(units = 32 , return_sequences = True))

In [0]:
model.add(GRU(units = 8,  return_sequences = True))

In [0]:
model.add(GRU(units = 4  ))

In [0]:
model.add(Dense (4 , activation = 'relu'))
model.add(Dense(1 , activation = 'sigmoid'))

In [0]:
optimizer = Adam(lr = 2e-3)

In [0]:
model.compile(loss = 'binary_crossentropy' , optimizer = optimizer , metrics = ['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 100)          1000000   
_________________________________________________________________
gru (GRU)                    (None, 544, 32)           12768     
_________________________________________________________________
gru_1 (GRU)                  (None, 544, 8)            984       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 1,013,933
Trainable params: 1,013,933
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train_pad , y_train , validation_split = 0.05 , epochs = 10 , batch_size = 32)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 23750 samples, validate on 1250 samples
Epoch 1/10
23750/23750 [==============================] - 667s 28ms/step - loss: 0.5219 - acc: 0.7213 - val_loss: 0.4409 - val_acc: 0.8152
Epoch 2/10
23750/23750 [==============================] - 666s 28ms/step - loss: 0.2299 - acc: 0.9129 - val_loss: 0.3746 - val_acc: 0.8376
Epoch 3/10
23750/23750 [==============================] - 666s 28ms/step - loss: 0.1304 - acc: 0.9549 - val_loss: 0.4378 - val_acc: 0.8320
Epoch 4/10
23750/23750 [==============================] - 665s 28ms/step - loss: 0.0671 - acc: 0.9807 - val_loss: 0.5668 - val_acc: 0.8288
Epoch 5/10
23750/23750 [==============================] - 666s 28ms/step - loss: 0.0458 - acc: 0.9874 - val_loss: 0.4320 - val_acc: 0.8760
Epoch 6/10
23750/23750 [==============================] - 656s 28ms/step - loss: 0.0361 - acc: 0.9909 - val_loss: 0.4141 - val_acc: 0.8920
Epoch 7/10
23750/23750 [==============================] - 664s 28ms/step - loss: 0.0309 - acc: 0.9923 - val_loss: 0.5

In [0]:
result = model.evaluate(x_test_pad , y_test)

NameError: ignored

In [0]:
print("Accuracy: {0: .2%}".format(result[1]))

NameError: ignored

In [0]:
%%time
y_pred = model.predict(x = x_test_pad[0:1000])
y_pred = y_pred.T[0]

CPU times: user 7.58 s, sys: 465 ms, total: 8.05 s
Wall time: 4.42 s


In [0]:
cls_pred = np.array([1.0 if p>0.5 else 0 for p in y_pred])

In [0]:
cls_true = np.array(y_test[0:1000])

In [0]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [0]:
len(incorrect)

228

In [0]:
idx = incorrect[0]
idx

11

In [0]:
text = x_test_text[idx]
text

'This film flopped miserably in the UK, and it didn\'t deserve to. The trailer of this film is slightly misleading, and I guess it mislead critics and audiences into thinking it was "Atonement: Part 2". While the film was marketed that way to capitalise on the earlier success of Joe Wright\'s BAFTA-winning film, it\'s very different in tone. It focuses on an imagination of sorts of Welsh poet Dylan Thomas\' life during the Second World War as the writer of propaganda films for the war effort, and his subsequent return to Wales. Director John Maybury quickly introduces Dylan\'s (Matthew Ryhs) childhood sweetheart Vera Phillips, played by Keira Knightley. She was Dylan\'s first love in their homeland, but the moment has passed, and singer Vera only wants it as a beautiful memory. Or does she? Vera unexpectedly strikes up a close bond with the other woman in Dylan\'s life, "Queen of Ireland, love of my life, mother of my child" Caitlin Thomas (Sienna Miller). The three form a sort of mena

In [0]:
y_pred[idx]

0.005081912

In [0]:
cls_pred[idx]

0.0

In [0]:
cls_true[idx]

1.0

In [0]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

In [0]:
tokens = tokenizer.texts_to_sequences(texts)
tokens

[[11, 17, 6, 799, 10, 62, 37, 9, 84, 9, 6, 34, 49],
 [49, 17],
 [273, 10, 37, 11, 17],
 [],
 [43, 10, 70, 3, 1940, 2188, 91, 11, 17, 233, 26, 49],
 [74, 17],
 [21, 3, 49, 17],
 [11, 17, 62, 1693, 67, 10, 76, 56, 290, 141, 592]]

In [0]:
tokens_pad = pad_sequences(tokens , maxlen = max_tokens , padding = pad , truncating = pad)
tokens_pad.shape

(8, 544)

In [0]:
model.predict(tokens_pad)

array([[0.9744879 ],
       [0.00955847],
       [0.07729653],
       [0.00602868],
       [0.00618783],
       [0.00962464],
       [0.01468459],
       [0.02814829]], dtype=float32)